In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import tensorflow as tf
import plotly.express as px
tf.keras.backend.clear_session()


2024-04-25 09:59:07.345594: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-25 09:59:07.368123: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 09:59:07.368138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 09:59:07.368701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 09:59:07.372631: I tensorflow/core/platform/cpu_feature_guar

In [2]:
MODELDIR = "../../models"

In [3]:
ENCODER_N = joblib.load("../data/NCRD/ncrd95-uniprot-ma_clf_enc.joblib")
ENCODER_N.categories_

/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.1.post1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[array(['NonR', 'aminoglycoside', 'beta-lactam', 'chloramphenicol',
        'glycopeptide', 'macrolide', 'macrolide-lincosamide-streptogramin',
        'multidrug', 'phosphonic acid', 'rifamycin', 'tetracycline'],
       dtype=object)]

In [4]:
ENCODER_P = joblib.load("../data/NCRD/ncrd95-ma_clf_enc.joblib")
ENCODER_P.categories_

[array(['aminoglycoside', 'beta-lactam', 'chloramphenicol', 'glycopeptide',
        'macrolide', 'macrolide-lincosamide-streptogramin', 'multidrug',
        'phosphonic acid', 'rifamycin', 'tetracycline'], dtype=object)]

In [5]:
def model_eval_n(model,data):
    yhat = model.predict(data)
    yhat_prob =[np.round(x[np.argmax(x)],3)  for x in yhat]
    yhat_oh = tf.convert_to_tensor([tf.one_hot(np.argmax(x),depth = len(ENCODER_N.categories_[0])) for x in yhat],dtype=tf.float32)
    classes = ENCODER_N.inverse_transform(yhat_oh).squeeze()
    return np.array(classes), np.array(yhat_prob)


def model_eval_p(model,data):
    yhat = model.predict(data)
    yhat_prob =[np.round(x[np.argmax(x)],3)  for x in yhat]
    yhat_oh = tf.convert_to_tensor([tf.one_hot(np.argmax(x),len(ENCODER_P.categories_[0])) for x in yhat],dtype=tf.float32)
    classes = ENCODER_P.inverse_transform(yhat_oh).squeeze()
    return np.array(classes), np.array(yhat_prob)

---

In [6]:
func = list("YNNYYNNNNNNNYNNNNNNNNNYNNNNNNNNNNNNYYNYYYYYNYNNNYN")

dantas_data = pd.read_csv("../data/real/dantas/hmm-hit.tsv",sep = "\t",names = ["Id","Acc","Sequence"])
dantas_data["Sequence"] = dantas_data.Sequence.apply(lambda x: " ".join(x))
dantas_data["Func"] = func
X_dantas = tf.convert_to_tensor(dantas_data.Sequence.to_list())

2024-04-25 09:59:08.348818: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 09:59:08.366237: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 09:59:08.366488: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
tf.keras.backend.clear_session()
cnn_ARPs = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD/cnn")
cnn_aligned = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-align/cnn")
cnn_unaligned = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign/cnn")
cnn_unaligned50 = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign50/cnn")
cnn_unaligned100 = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign100/cnn")

tf.keras.backend.clear_session()
yhat = model_eval_p(cnn_ARPs,X_dantas)
yhat_align = model_eval_n(cnn_aligned, X_dantas)
yhat_unalign = model_eval_n(cnn_unaligned, X_dantas)
yhat_unalign50 = model_eval_n(cnn_unaligned50,X_dantas)
yhat_unalign100 = model_eval_n(cnn_unaligned100,X_dantas)

dantas_data["CNN-Only-ARPs-class"] = yhat[0]
dantas_data["CNN-Only-ARPs-prob"] = yhat[1]
dantas_data["CNN-Only-ARPs-class"] = dantas_data["CNN-Only-ARPs-class"].where(dantas_data["CNN-Only-ARPs-prob"] > 0.5, "NonR").to_list()

dantas_data["CNN-Aligned-class"] = yhat_align[0]
dantas_data["CNN-Aligned-prob"] = yhat_align[1]
dantas_data["CNN-Unaligned-class"] = yhat_unalign[0]
dantas_data["CNN-Unaligned-prob"] = yhat_unalign[1]
# dantas_data["CNN-Unaligned50-class"] = yhat_unalign50[0]
# dantas_data["CNN-Unaligned50-prob"] = yhat_unalign50[1]
# dantas_data["CNN-Unaligned100-class"] = yhat_unalign100[0]
# dantas_data["CNN-Unaligned100-prob"] = yhat_unalign100[1]

2024-04-25 09:59:09.718983: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-25 09:59:09.967863: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-04-25 09:59:10.032588: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


2/2 [==============================] - 0s 9ms/step


In [8]:
tf.keras.backend.clear_session()
cnn_ARPs = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD/cnn-same")
cnn_aligned = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-align/cnn-same")
cnn_unaligned = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign/cnn-same")
cnn_unaligned50 = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign50/cnn-same")
cnn_unaligned100 = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign100/cnn-same")

tf.keras.backend.clear_session()
yhat = model_eval_p(cnn_ARPs,X_dantas)
yhat_align = model_eval_n(cnn_aligned, X_dantas)
yhat_unalign = model_eval_n(cnn_unaligned, X_dantas)
yhat_unalign50 = model_eval_n(cnn_unaligned50,X_dantas)
yhat_unalign100 = model_eval_n(cnn_unaligned100,X_dantas)

dantas_data["CNNS-Only-ARPs-class"] = yhat[0]
dantas_data["CNNS-Only-ARPs-prob"] = yhat[1]
dantas_data["CNNS-Only-ARPs-class"] = dantas_data["CNNS-Only-ARPs-class"].where(dantas_data["CNNS-Only-ARPs-prob"] > 0.5, "NonR").to_list()

dantas_data["CNNS-Aligned-class"] = yhat_align[0]
dantas_data["CNNS-Aligned-prob"] = yhat_align[1]
dantas_data["CNNS-Unaligned-class"] = yhat_unalign[0]
dantas_data["CNNS-Unaligned-prob"] = yhat_unalign[1]
# dantas_data["CNNS-Unaligned50-class"] = yhat_unalign50[0]
# dantas_data["CNNS-Unaligned50-prob"] = yhat_unalign50[1]
# dantas_data["CNNS-Unaligned100-class"] = yhat_unalign100[0]
# dantas_data["CNNS-Unaligned100-prob"] = yhat_unalign100[1]

2024-04-25 09:59:14.169443: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 714.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-25 09:59:14.485659: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 714.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-25 09:59:14.814922: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 714.12MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-25 09:59:15.138754: W external/local_tsl/tsl/framewor

1/2 [==============>...............] - ETA: 2s

2024-04-25 09:59:15.798766: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 713.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


2/2 [==============================] - 0s 26ms/step


In [9]:
tf.keras.backend.clear_session()
cnn_ARPs = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD/gru")
cnn_aligned = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-align/gru")
cnn_unaligned = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign/gru")
cnn_unaligned50 = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign50/gru")
cnn_unaligned100 = tf.keras.models.load_model(f"{MODELDIR}/Fase5-NCRD-neg-unalign100/gru")

tf.keras.backend.clear_session()
yhat = model_eval_p(cnn_ARPs,X_dantas)
yhat_align = model_eval_n(cnn_aligned, X_dantas)
yhat_unalign = model_eval_n(cnn_unaligned, X_dantas)
yhat_unalign50 = model_eval_n(cnn_unaligned50,X_dantas)
yhat_unalign100 = model_eval_n(cnn_unaligned100,X_dantas)

dantas_data["GRU-Only-ARPs-class"] = yhat[0]
dantas_data["GRU-Only-ARPs-prob"] = yhat[1]
dantas_data["GRU-Only-ARPs-class"] = dantas_data["GRU-Only-ARPs-class"].where(dantas_data["GRU-Only-ARPs-prob"] > 0.5, "NonR").to_list()

dantas_data["GRU-Aligned-class"] = yhat_align[0]
dantas_data["GRU-Aligned-prob"] = yhat_align[1]
dantas_data["GRU-Unaligned-class"] = yhat_unalign[0]
dantas_data["GRU-Unaligned-prob"] = yhat_unalign[1]
# dantas_data["GRU-Unaligned50-class"] = yhat_unalign50[0]
# dantas_data["GRU-Unaligned50-prob"] = yhat_unalign50[1]
# dantas_data["GRU-Unaligned100-class"] = yhat_unalign100[0]
# dantas_data["GRU-Unaligned100-prob"] = yhat_unalign100[1]

2024-04-25 09:59:18.385716: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-25 09:59:18.392072: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-25 09:59:18.470264: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-25 09:59:18.474753: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond' has 4 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-25 09:59:18.636137: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _ou

InternalError: Graph execution error:

Detected at node cond/CudnnRNNV3 defined at (most recent call last):
  File "/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/tiago/anaconda3/envs/deepsea-project/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/tiago/anaconda3/envs/dee

In [ ]:
dantas_data.head()

In [ ]:
ncrd_dantas_diamond = pd.read_csv("../data/real/dantas/diamond/ncrd-dantas-out.tsv", sep = "\t", names = ["qseqid","qlen","sseqid","slen","evalue", "bitscore", "length", "pident"])
ncrd_dantas_diamond = ncrd_dantas_diamond.rename(columns={"qseqid":"Id"})
ncrd_dantas_diamond.tail(3)

In [ ]:
dantas_ncrd_data = dantas_data.merge(ncrd_dantas_diamond[["Id","pident"]], on = "Id", how = "left")

In [ ]:
dantas_amrfinder = pd.read_csv("../data/real/dantas/hmm-amrfinder-out.csv", sep = "\t").rename(columns= {"Protein identifier":"Id"})

In [ ]:
dantas_ncrd_amrfinder = dantas_ncrd_data.merge(dantas_amrfinder[["Id","Class","% Identity to reference sequence","Method"]], on = "Id", how = "left")

In [ ]:
dantas_rgi = pd.read_csv("../data/real/dantas/hmm-rgi.txt",sep = "\t").rename(columns= {"ORF_ID":"Id"})

In [ ]:
dantas_ncrd_amrfinder_rgi = dantas_ncrd_amrfinder.merge(dantas_rgi[["Id","Drug Class","Best_Identities"]], on = "Id", how = "left")

In [ ]:
dantas_deeparg = pd.read_csv("../data/DeepARG/hmm-hits.mapping.ARG", sep = "\t").rename(columns = {"read_id":"Id"})

In [ ]:
dantas_final = dantas_ncrd_amrfinder_rgi.merge(dantas_deeparg[["Id","predicted_ARG-class","probability","identity"]], on = "Id", how = "left")

In [ ]:
names = {
    # "Only ARGs": "DeepSEA-OnlyARGs" ,
    # "Aligned": "DeepSEA-Aligned",	
    # 'Unaligned': "DeepSEA-Unaligned",	
    # "Unaligned50": "DeepSEA-Unaligned50",	
    # "Unaligned100": "DeepSEA-Unaligned100",
    # "pident": "DeepSEA-Ident",
    "Class": "AMRFinder-class",
    "% Identity to reference sequence": "AMRfinder-Ident",
    "Method": "AMRFinder-Method",
    "Drug Class":"RGI-class",
    "Best_Identities": "RGI-Ident",
    "predicted_ARG-class": "DeepARG-class",
    "probability": "DeepARG-Prob",
    "identity": "DeepARG-Ident"
}

dantas_final = dantas_final.rename(columns=names)
dantas_final["AMRFinder-class"] = dantas_final["AMRFinder-class"].str.lower()
dantas_final["RGI-class"].replace({"tetracycline antibiotic":"tetracycline","glycylcycline; tetracycline antibiotic":"tetracycline"}, inplace = True)
dantas_final.head()

In [ ]:
sankey = pd.concat([
    dantas_final.loc[:,dantas_final.columns.str.contains("class")],
    dantas_final[["Func","Id"]]
], axis = 1)



In [ ]:
classes = ["tetracycline","beta-lactam","multidrug","glycopeptide","macrolide-lincosamide-streptogramin","aminoglycoside"]
id = sankey.Id.to_list()
func = sankey.Func.to_list()

In [ ]:
sankey.fillna("NonR").replace({"NonR"}).drop(["CNN-Only-ARPs-class","CNNS-Only-ARPs-class","GRU-Only-ARPs-class"], axis=1).melt(id_vars = "Id")

In [ ]:
import plotly.graph_objects as go

In [ ]:
dantas_final_positive_ds = dantas_final.loc[dantas_final.Func == "Y", dantas_final.columns.str.contains("class")].melt()
dantas_final_positive_ds = pd.crosstab(index = dantas_final_positive_ds["variable"],columns = dantas_final_positive_ds["value"])
dantas_final_positive_ds["tetracycline"] = dantas_final_positive_ds["tetracycline"].div(len(dantas_final[dantas_final.Func == "Y"])).mul(100)
dantas_final_positive_ds = dantas_final_positive_ds[["tetracycline"]]
dantas_final_positive_ds.index.name = "Tool"
dantas_final_positive_ds.reset_index(inplace = True)
dantas_final_positive_ds.columns.name = None

In [ ]:
dantas_final_positive_ds["Tag"] = ["Other","DeepSEA","DeepSEA","DeepSEA","DeepSEA","DeepSEA","DeepSEA","DeepSEA","DeepSEA","DeepSEA","DeepSEA","Other","DeepSEA","DeepSEA","DeepSEA","DeepSEA","DeepSEA","Other"]

In [ ]:
dantas_final_positive_ds["Tool"] = dantas_final_positive_ds["Tool"].str.rstrip("-class")

In [ ]:
dantas_final_positive_ds

In [ ]:
fig = px.histogram(
    dantas_final_positive_ds,
    x = "Tool",
    y = "tetracycline",
    color = "Tag",
    barmode = "group",
)

fig.show()